**Importing the Libraries**

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

**Step 1: Loading the Dataset**

In [2]:
# Load the data
data = pd.read_csv('/content/airline-passengers - Sheet1.csv')

**Step 2: Data Preprocessing**

In [3]:
# Convert the data to a numpy array
data_array = data['Passengers'].values

# Normalize the data
data_array = data_array / 1000.0

In [4]:
# Split the data into training and testing sets
train_size = int(0.8 * len(data_array))
train_data, test_data = data_array[0:train_size], data_array[train_size:len(data_array)]

In [5]:
# Create a function to create sequences from the data
def create_sequences(data, seq_len):
    sequences = []
    for i in range(len(data) - seq_len):
        sequences.append(data[i:i+seq_len])
    return np.array(sequences)

In [6]:
# Create sequences from the training and testing data
seq_len = 12
train_sequences = create_sequences(train_data, seq_len)
test_sequences = create_sequences(test_data, seq_len)

In [7]:
# Create a custom dataset class for our sequences
class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        x = sequence[:-1]
        y = sequence[-1]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [8]:
# Create data loaders for the training and testing data
batch_size = 32
train_dataset = SequenceDataset(train_sequences)
test_dataset = SequenceDataset(test_sequences)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

**Step 3: Create the LSTM Model**

In [9]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [10]:
# Initialize the model, loss function, and optimizer
model = LSTMModel(input_dim=1, hidden_dim=50, output_dim=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

**Step 4: Train The Model**

In [11]:
# Train the model
for epoch in range(100):
    for x, y in train_loader:
        x = x.unsqueeze(2)
        y = y.unsqueeze(1)
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.005805084947496653
Epoch 2, Loss: 0.003412355203181505
Epoch 3, Loss: 0.0060778516344726086
Epoch 4, Loss: 0.0014699731254950166
Epoch 5, Loss: 0.007481544278562069
Epoch 6, Loss: 0.0018805249128490686
Epoch 7, Loss: 0.006202310789376497
Epoch 8, Loss: 0.0032763034105300903
Epoch 9, Loss: 0.006707929074764252
Epoch 10, Loss: 0.0026396499015390873
Epoch 11, Loss: 0.0016445403452962637
Epoch 12, Loss: 0.003928287886083126
Epoch 13, Loss: 0.006303322501480579
Epoch 14, Loss: 0.0011638139840215445
Epoch 15, Loss: 0.002622112398967147
Epoch 16, Loss: 0.0024443145375698805
Epoch 17, Loss: 0.0037794082891196012
Epoch 18, Loss: 0.002289674011990428
Epoch 19, Loss: 0.004236054141074419
Epoch 20, Loss: 0.001733946381136775
Epoch 21, Loss: 0.003110318211838603
Epoch 22, Loss: 0.0025738514959812164
Epoch 23, Loss: 0.002754020970314741
Epoch 24, Loss: 0.005498877260833979
Epoch 25, Loss: 0.0021865335293114185
Epoch 26, Loss: 0.0028568808920681477
Epoch 27, Loss: 0.00251502380706369

**Step 5: Evaluating Model**

In [12]:
# Evaluate the model
model.eval()
test_loss = 0
with torch.no_grad():
    for x, y in test_loader:
        x = x.unsqueeze(2)
        y = y.unsqueeze(1)
        output = model(x)
        loss = criterion(output, y)
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss}')

Test Loss: 0.003033713437616825


In [13]:
# Use the model to predict the next sequence of numbers
def predict_next_sequence(model, sequence, seq_len):
    predicted_sequence = []
    current_sequence = torch.tensor(sequence[-seq_len:], dtype=torch.float32).unsqueeze(0).unsqueeze(2)

    for i in range(12):
        with torch.no_grad():
            output = model(current_sequence)
            predicted_value = output.item()
            predicted_sequence.append(predicted_value)

            # Update the current sequence to include the predicted value
            current_sequence = torch.cat([current_sequence[:, 1:, :], torch.tensor([[[predicted_value]]], dtype=torch.float32)], dim=1)

    return predicted_sequence

**Step 6: Final Result**

In [14]:
# Get the predicted sequence
predicted_sequence = predict_next_sequence(model, data_array, seq_len)

print('Predicted Next Sequence:', predicted_sequence)

Predicted Next Sequence: [0.3849259614944458, 0.3904801607131958, 0.39612001180648804, 0.3974493741989136, 0.39933568239212036, 0.3995046615600586, 0.3980133533477783, 0.394961953163147, 0.39221280813217163, 0.3911033272743225, 0.3923373222351074, 0.39604806900024414]
